# XgBoost Classifier with optuna

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
def objective(trial):
    (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        
        "tree_method": "exact",
        
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
      
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
       
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

[I 2023-05-23 04:27:51,851] A new study created in memory with name: no-name-1f868bdb-1789-4cb3-901c-9fc03d72016c
[I 2023-05-23 04:27:51,871] Trial 0 finished with value: 0.8951048951048951 and parameters: {'booster': 'gblinear', 'lambda': 4.319112994536401e-07, 'alpha': 0.4259221418464674, 'subsample': 0.7143229521517696, 'colsample_bytree': 0.5107788529729906}. Best is trial 0 with value: 0.8951048951048951.
[I 2023-05-23 04:27:51,896] Trial 1 finished with value: 0.9300699300699301 and parameters: {'booster': 'dart', 'lambda': 1.048037136142059e-05, 'alpha': 0.08744729409518917, 'subsample': 0.8463034976936525, 'colsample_bytree': 0.5067968208215066, 'max_depth': 5, 'min_child_weight': 8, 'eta': 3.911015969197081e-06, 'gamma': 2.1156826837947285e-07, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 3.065003107165861e-05, 'skip_drop': 3.686310144682328e-06}. Best is trial 1 with value: 0.9300699300699301.
[I 2023-05-23 04:27:51,916] Tria

Number of finished trials:  100
Best trial:
  Value: 0.986013986013986
  Params: 
    booster: dart
    lambda: 9.761767167420675e-07
    alpha: 0.008914308651798606
    subsample: 0.9007218680870888
    colsample_bytree: 0.4483274586850436
    max_depth: 3
    min_child_weight: 4
    eta: 3.930229538196082e-08
    gamma: 0.3548755204905699
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.0007792909942270239
    skip_drop: 1.60577509166331e-08


# Differences between Optuna, Grid Search and Random Search

1. GridSearch looks at every possible combination of hyperparameters, RandomSearch selects and tests a random combination of hyperparameters whereas Optuna is an automatic hyperparameter optimization software framework, it automatically searches and finds optimal hyperparameter values by trial and error for excellent performance. 

2. GridSearch and RandomSearch are more straight-forward method that can be effective for small search spaces, Optuna is a more advanced and efficient method that is better suited for larger and more complex search spaces.

3. Optuna is more faster and efficient than GridSearch and RandomSearch as it uses Bayesian optimization algorithmalgorithm. 

4. Optuna offers a variety of search algorithms and pruning techniques that can further improve the optimization process, which is not there in GridSearch and RandomSearch.

5. Optuna can also handle conditional hyperparameters, which are hyperparameters whose values depend on the values of other hyperparameters, making it more flexible and versatile than GridSearch and RandomSearch.